# **Surprisal**


Some reference:
- https://www.kaggle.com/code/smidgin/surprisal-with-gpt-2
- https://pypi.org/project/pysurprisal/
- https://github.com/aalok-sathe/surprisal
- https://github.com/byungdoh/slm_surprisal
- https://github.com/byungdoh/llm_surprisal
- https://github.com/tmalsburg/llm_surprisal
- https://github.com/benedict-krieger/llm-surprisal-rerps

Where I lerarned:
- https://huggingface.co/learn/llm-course/it/




Prima importiamo il modello.

- ***clean_up_tokenization_spaces*** = indica al 'tokenizer0 di 'ripulire' gli spazi quando decodifica i token in testo. Corregge artefatti di tokenizzazione come spazi prima della punteggiatura, doppi spazi. Può essere messo _False_ se si vuole analizzare il testo così com'è.

- ***from IPython.display import clear_output; clear_output()*** = serve a pulire l’output della cella in un notebook Jupyter/IPython. Lo si usa spesso dopo il caricamento del modello per togliere log, warning, o barre di progresso, lasciando il notebook più “pulito” e leggibile. In uno script Python normale (non notebook) non ha effetto e non è necessario.


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

device= torch.device("cuda" if torch.cuda.is_available() else 'cpu')

model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
toeknizer = GPT2LMHeadModel.from_pretrained("gpt2", clean_up_tokenization_space = True)
from IPython.display import clear_output; clear_output()

Calcoliamo le metriche

import torch
from torch import Tensor
import torch.nn.functional as D
import ml_utils as mu

def get_token_metrics (texts: list[str], model: GPT2LMHeadModel, tokenizer: GPT2Tokenizer, truncation=False) -> dict:
    if tokenizer.pad_token is None:
        tolenizer.pad_token = tokenizer.eos_token

    B = len(texts)
    device = next(model.parameters()).device

    inputs = tokenizer(texts, return_tensors = "pt", padding=True, truncation = truncation, return_length =True).to(device)

    